In [1]:
import pandas as pd
import numpy as np

In [2]:
x_tr_title_id = np.load('E:/qrf/Lab Stuff/newDataset/x_tr_title_id.npy')
x_vl_title_id = np.load('E:/qrf/Lab Stuff/newDataset/x_vl_title_id.npy')
x_tr_body_id = np.load('E:/qrf/Lab Stuff/newDataset/x_tr_body_id.npy')
x_vl_body_id = np.load('E:/qrf/Lab Stuff/newDataset/x_vl_body_id.npy')
y_tr = np.load('E:/qrf/Lab Stuff/newDataset/y_tr.npy')
y_vl = np.load('E:/qrf/Lab Stuff/newDataset/y_vl.npy')

In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals
import collections
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)
from tensorflow.compat.v1.keras.layers import CuDNNGRU
from tensorflow.keras.layers import Embedding, Dropout, LSTM, Dense, Bidirectional, Flatten
from tensorflow.keras.optimizers import Adam

in_title = keras.Input(batch_shape=(None, x_tr_title_id.shape[1]))
in_body = keras.Input(batch_shape=(None, x_tr_body_id.shape[1]))


#title
embeddedT = keras.layers.Embedding(10000, 5)(in_title)
averagedT = tf.reduce_mean(embeddedT, axis=1)
lstmT = Bidirectional(LSTM(32, return_sequences = True))(embeddedT)
flattenT = Flatten()(lstmT)
fcT = Dense(32, activation='relu')(flattenT)


#body
embeddedB = keras.layers.Embedding(10000, 5)(in_body)
averagedB = tf.reduce_mean(embeddedB, axis=1)
lstmB = Bidirectional(LSTM(32, return_sequences = True))(embeddedB)
flattenB = Flatten()(lstmB)
fcB = Dense(32, activation='relu')(flattenB)


#fused
fused = tf.concat([fcT, fcB], axis=-1)
p_stackOverflow = Dense(1, activation='sigmoid')(fused)


model = keras.Model(
    inputs={
        'title': in_title,
        'body': in_body,
        },
        outputs=p_stackOverflow
)

model.compile(
    optimizer=Adam(),
    loss='binary_crossentropy',
    metrics=['accuracy', 'AUC']
)

model.summary()

2.6.0
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 100, 5)       50000       input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 100, 5)       50000       input_2[0][0]                    
________________________________________________________________________________________

In [4]:
history = model.fit(
    
    x={
        'title': x_tr_title_id,
        'body': x_tr_body_id,
    },
    
    y=y_tr,
    
    epochs=10,
    batch_size=128,
    
    validation_data=({'title': x_vl_title_id,
                      'body': x_vl_body_id,
                     },
                     
                     y_vl),
    
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)
    ],

    verbose=1
)

Epoch 1/10
2021/2021 [==============================] - 463s 225ms/step - loss: 0.2376 - accuracy: 0.9054 - auc: 0.9650 - val_loss: 0.2048 - val_accuracy: 0.9206 - val_auc: 0.9734
Epoch 2/10
2021/2021 [==============================] - 448s 221ms/step - loss: 0.1900 - accuracy: 0.9277 - auc: 0.9770 - val_loss: 0.2028 - val_accuracy: 0.9214 - val_auc: 0.9746c: 0.977 - ETA: 39s - loss: 0.1898 - accuracy: 0.9277 - auc - ETA: 38s - loss: 0.1897 - - ETA: 31s - loss: 0.1898 - accuracy: 0.9277 - auc:  - ETA: 30s - loss: 0. - ETA: 4s - los
Epoch 3/10
2021/2021 [==============================] - 426s 211ms/step - loss: 0.1717 - accuracy: 0.9351 - auc: 0.9810 - val_loss: 0.1990 - val_accuracy: 0.9234 - val_auc: 0.9752
Epoch 4/10
2021/2021 [==============================] - 410s 203ms/step - loss: 0.1535 - accuracy: 0.9423 - auc: 0.9847 - val_loss: 0.2046 - val_accuracy: 0.9211 - val_auc: 0.9743
Epoch 5/10
2021/2021 [==============================] - 442s 219ms/step - loss: 0.1360 - accuracy: 0.9